In [46]:
import numpy as np
import pandas as pd
from imblearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.compose import ColumnTransformer
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import NearMiss
from fairlearn.metrics import MetricFrame, equalized_odds_difference
from sklearn.metrics import roc_auc_score
import warnings
from sklearn.exceptions import ConvergenceWarning
from joblib import Parallel, delayed
from jupyter_server.services.config import ConfigManager
import os

cm = ConfigManager()
cm.update('notebook', {"ServerApp.iopub_msg_rate_limit": 100000})

warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=ConvergenceWarning)
%run helper.ipynb

In [33]:
print("2")

2


In [37]:
# x, y = premature(add_features_df, term_info_df, False)
# print(y)

In [89]:
add_features_df = pd.read_csv("add_features.csv", low_memory=False)

In [ ]:
# graduation_multiverse_col_names = ["Include_Transfer", "Include_Sex", "Include_Race",  "Train_Size", "Handle_Nan", 
#                                "Scaler", "Encoder", "Sampler", "Classifier", "N_Estimators", "Learning_Rate", "C", "AUC_Graduated_In_4", 
#                                 "AUC_Graduated_In_6", "Equalized_Odds_Difference_Sex_4", "Equalized_Odds_Difference_Race_4", "Equalized_Odds_Difference_FG_4", "Equalized_Odds_Difference_Income_4",  "Equalized_Odds_Difference_Sex/Race_4", "Equalized_Odds_Difference_FG/Income_4",
#                                  "Equalized_Odds_Difference_Sex_6", "Equalized_Odds_Difference_Race_6", "Equalized_Odds_Difference_FG_6", "Equalized_Odds_Difference_Income_6", "Equalized_Odds_Difference_Sex/Race_6",
#                                 "Equalized_Odds_Difference_FG/Income_6"]
# graduation_multiverse_df = pd.DataFrame(columns=reenroll_multiverse_col_names)    

In [90]:
graduation_multiverse_col_names = ["Include_Transfer", "Include_Sex", "Include_Race",  "Train_Size", "Handle_Nan", 
                               "Scaler", "Encoder", "Sampler", "Classifier", "N_Estimators", "Learning_Rate", "C", "AUC_Graduated_In_4", 
                                "AUC_Graduated_In_6", "Equalized_Odds_Difference_Sex_4", "Equalized_Odds_Difference_Race_4", "Equalized_Odds_Difference_FG_4", "Equalized_Odds_Difference_Income_4",  "Equalized_Odds_Difference_Sex/Race_4", "Equalized_Odds_Difference_FG/Income_4",
                                 "Equalized_Odds_Difference_Sex_6", "Equalized_Odds_Difference_Race_6", "Equalized_Odds_Difference_FG_6", "Equalized_Odds_Difference_Income_6", "Equalized_Odds_Difference_Sex/Race_6",
                                "Equalized_Odds_Difference_FG/Income_6"]
feature_columns = ['STDNT_INTL_IND', 'STDNT_NTV_ENG_SPKR_IND', 'FIRST_US_PRMNNT_RES_PSTL_5_CD', 'PRNT_MAX_ED_LVL_DES', 'FIRST_TERM_ATTND_SHORT_DES',
                      'STARTING_AGE', 'IS_FIRST_GEN', 'TRANSFER_CREDITS', 'MAX_SAT_SCR', 'EST_GROSS_FAM_INC_DES', 'Missing_Income', 'FIRST_YR_CUM_GPA', 
                      'FIRST_YR_TAKEN_CREDITS', 'HS_GPA', 'HS_CALC_IND', 'HS_CHEM_LAB_IND', 'PGM_1_MAJOR_1_CIP_DES', 'FIRST_UG_ENTRY_TYP_DES', 'STDNT_SEX_SHORT_DES',
                  'STDNT_ETHNC_GRP_SHORT_DES', 'Sex/Race', 'FG/Income']

# existing_df = pd.read_csv('partial_results.csv', low_memory=False)


def process_grad_combination(include_transfer, handle_nan, train_size, include_sex, include_race, encoder, scaler, sampler, classifier, classifier_params):
    pipe = Pipeline(steps=[
        ("Transformer", 'passthrough'),
        ("Scaler", 'passthrough'),
        ("Sampler", 'passthrough'),
        ("Classifier", 'passthrough')
    ]) 
    
    
#     decision_df = pd.DataFrame([verse_dict])
#     common_cols = decision_df.columns;
#     # Function to compare rows with proper NaN handling
#     def compare_rows(row1, row2):
#         # Compare row1 and row2, treating NaNs as equal
#         comparison = (row1 == row2) | ((row1.isna()) & (row2.isna()))
#         return comparison.all()

#     # Check each row in existing_df to see if it matches the current row in shit
#     is_duplicate = existing_df[common_cols].apply(lambda row: compare_rows(row, decision_df.iloc[0][common_cols]), axis=1)

#     if is_duplicate.any():
#         print("Duplicate entry found, extracting and returning the existing result...")
#         duplicate_row = existing_df.loc[is_duplicate].iloc[0]
#         # print(duplicate_row.to_dict())
#         return duplicate_row.to_dict()

    # Load and preprocess data
    premature_X, premature_Y = graduation_premature(add_features_df, include_transfer=include_transfer)
    features_df = premature_X[feature_columns].reset_index(drop=True)
    premature_Y = premature_Y.reset_index(drop=True)
    
    if handle_nan == 'Drop':
        rows_with_nan = features_df.isnull().any(axis=1)
        X_after_nan = features_df[~rows_with_nan]
        Y_after_nan = premature_Y[~rows_with_nan]
        X_train, X_test, Y_train, Y_test = train_test_split(X_after_nan, Y_after_nan, train_size=train_size) # Train-Test Split
        X_train = X_train.reset_index(drop=True)
        X_test = X_test.reset_index(drop=True)
        Y_train = Y_train.reset_index(drop=True)
        Y_test = Y_test.reset_index(drop=True)
        Y_train_4 = Y_train['GRADUATED_IN_4']
        Y_train_6 = Y_train['GRADUATED_IN_6']

        Y_test_4 = Y_test['GRADUATED_IN_4']
        Y_test_6 = Y_test['GRADUATED_IN_6']
    else:  
        X_train, X_test, Y_train, Y_test = train_test_split(features_df, premature_Y, train_size=train_size) # Train - Test Split first
        X_train = X_train.reset_index(drop=True)
        X_test = X_test.reset_index(drop=True)
        Y_train = Y_train.reset_index(drop=True)
        Y_test = Y_test.reset_index(drop=True)
        
        Y_train_4 = Y_train['GRADUATED_IN_4']
        Y_train_6 = Y_train['GRADUATED_IN_6']

        Y_test_4 = Y_test['GRADUATED_IN_4']
        Y_test_6 = Y_test['GRADUATED_IN_6']

        # Perform imputation on the training data and apply it to the test data
        imputer = SimpleImputer(strategy="most_frequent")
        X_train_array = imputer.fit_transform(X_train)
        X_train = pd.DataFrame(X_train_array, columns=X_train.columns, index=X_train.index)

        X_test_array = imputer.transform(X_test)
        X_test = pd.DataFrame(X_test_array, columns=X_test.columns, index=X_test.index)

    sex_test = X_test['STDNT_SEX_SHORT_DES']
    race_test = X_test['STDNT_ETHNC_GRP_SHORT_DES']
    sex_race_test = X_test['Sex/Race']
    fg_test = X_test['IS_FIRST_GEN']
    income_test = X_test['EST_GROSS_FAM_INC_DES']
    fg_income_test = X_test['FG/Income']
    
    X_train = X_train.drop(columns=['Sex/Race', 'FG/Income'])
    X_test = X_test.drop(columns=['Sex/Race', 'FG/Income'])
    
    # Handle categorical features
    if not include_sex:
        X_train = X_train.drop(columns=['STDNT_SEX_SHORT_DES'])
        X_test = X_test.drop(columns=['STDNT_SEX_SHORT_DES'])
    if not include_race:
        X_train = X_train.drop(columns=['STDNT_ETHNC_GRP_SHORT_DES'])
        X_test = X_test.drop(columns=['STDNT_ETHNC_GRP_SHORT_DES'])

    # Setup pipeline with caching
    categorical_columns = X_train.select_dtypes(include=['object']).columns
    preprocessor = ColumnTransformer(
        transformers=[
            ('cat', encoder, categorical_columns)
        ],
        remainder='passthrough'
    )
    
    
    classifier.set_params(**classifier_params)
    pipe.set_params(**{
        "Transformer": preprocessor,
        "Scaler": scaler,
        "Sampler": sampler,
        "Classifier": classifier
    })


    # Fit and evaluate
    pipe.fit(X_train, Y_train_4)
    y_pred_4 = pipe.predict(X_test)
    y_prob_4 = pipe.predict_proba(X_test)[:, 1]

    # Metrics
    auc_4 = roc_auc_score(Y_test_4, y_prob_4)
    sex_eq_odds_diff_4 = equalized_odds_difference(Y_test_4, y_pred_4, sensitive_features=sex_test)
    race_eq_odds_diff_4 = equalized_odds_difference(Y_test_4, y_pred_4, sensitive_features=race_test)
    fg_eq_odds_diff_4 = equalized_odds_difference(Y_test_4, y_pred_4, sensitive_features=fg_test)
    income_eq_odds_diff_4 = equalized_odds_difference(Y_test_4, y_pred_4, sensitive_features=income_test)
    sex_race_eq_odds_diff_4 = equalized_odds_difference(Y_test_4, y_pred_4, sensitive_features=sex_race_test)
    fg_income_eq_odds_diff_4 = equalized_odds_difference(Y_test_4, y_pred_4, sensitive_features=fg_income_test)
    
    # Fit and evaluate
    pipe.fit(X_train, Y_train_6)
    y_pred_6 = pipe.predict(X_test)
    y_prob_6 = pipe.predict_proba(X_test)[:, 1]

    # Metrics
    auc_6 = roc_auc_score(Y_test_6, y_prob_6)
    sex_eq_odds_diff_6 = equalized_odds_difference(Y_test_6, y_pred_6, sensitive_features=sex_test)
    race_eq_odds_diff_6 = equalized_odds_difference(Y_test_6, y_pred_6, sensitive_features=race_test)
    fg_eq_odds_diff_6 = equalized_odds_difference(Y_test_6, y_pred_6, sensitive_features=fg_test)
    income_eq_odds_diff_6 = equalized_odds_difference(Y_test_6, y_pred_6, sensitive_features=income_test)
    sex_race_eq_odds_diff_6 = equalized_odds_difference(Y_test_6, y_pred_6, sensitive_features=sex_race_test)
    fg_income_eq_odds_diff_6 = equalized_odds_difference(Y_test_6, y_pred_6, sensitive_features=fg_income_test)
    
    
    
    verse_dict = {
    "Include_Transfer": 1 if include_transfer else 0,
    "Handle_Nan": handle_nan,
    "Train_Size": train_size,
    "Include_Sex": 1 if include_sex else 0,
    "Include_Race": 1 if include_race else 0,
    "Encoder": 'OneHotEncoder' if isinstance(encoder, OneHotEncoder) else 'OrdinalEncoder',
    "Scaler": 'StandardScaler' if scaler else None,
    "Sampler": sampler.__class__.__name__ if sampler else None,
    "Classifier": classifier.__class__.__name__,
    }

    if isinstance(classifier, RandomForestClassifier):
        verse_dict['N_Estimators'] = classifier.n_estimators
    elif isinstance(classifier, GradientBoostingClassifier):
        verse_dict['Learning_Rate'] = classifier.learning_rate
    elif isinstance(classifier, LogisticRegression):
        verse_dict['C'] = classifier.C


    # Add the AUC and equalized odds difference after classifier parameters
    verse_dict.update({
        "AUC_Graduated_In_4": auc_4,
        "Equalized_Odds_Difference_Sex_4": sex_eq_odds_diff_4,
        "Equalized_Odds_Difference_Race_4": race_eq_odds_diff_4,
        "Equalized_Odds_Difference_FG_4": fg_eq_odds_diff_4,
        "Equalized_Odds_Difference_Income_4": income_eq_odds_diff_4,
        "Equalized_Odds_Difference_Sex/Race_4": sex_race_eq_odds_diff_4,
        "Equalized_Odds_Difference_FG/Income_4": fg_income_eq_odds_diff_4,
        "AUC_Graduated_In_6": auc_6,
        "Equalized_Odds_Difference_Sex_6": sex_eq_odds_diff_6,
        "Equalized_Odds_Difference_Race_6": race_eq_odds_diff_6,
        "Equalized_Odds_Difference_FG_6": fg_eq_odds_diff_6,
        "Equalized_Odds_Difference_Income_6": income_eq_odds_diff_6,
        "Equalized_Odds_Difference_Sex/Race_6": sex_race_eq_odds_diff_6,
        "Equalized_Odds_Difference_FG/Income_6": fg_income_eq_odds_diff_6
    })
    
    # Ensure that the DataFrame has consistent columns
    result_df = pd.DataFrame([verse_dict], columns=graduation_multiverse_col_names)
    save_header = not os.path.isfile('graduation_partial_results.csv')

    # Save the result after processing each combination
    with open('graduation_partial_results.csv', 'a') as f:
        result_df.to_csv(f, header=save_header, index=False)
    return verse_dict

# Parallel execution
results = Parallel(n_jobs=-1, backend="multiprocessing")(
    delayed(process_grad_combination)(
        include_transfer, handle_nan, train_size, include_sex, include_race, encoder, scaler, sampler, classifier, classifier_params
    )
   for include_transfer in [True, False]
    for handle_nan in ["Impute", "Drop"]
    for train_size in [0.7, 0.8]
    for include_sex in [True, False]
    for include_race in [True, False]
    for encoder in [OneHotEncoder(handle_unknown='ignore'), OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)]
    for scaler in [StandardScaler(with_mean=False), None]
    for sampler in [SMOTE(), NearMiss(), None]
    for classifier, classifier_params in [
        (RandomForestClassifier(), {"n_estimators": 50}),
        (RandomForestClassifier(), {"n_estimators": 100}),
        (RandomForestClassifier(), {"n_estimators": 150}),
        (GradientBoostingClassifier(), {"learning_rate": 0.01}),
        (GradientBoostingClassifier(), {"learning_rate": 0.1}),
        (GradientBoostingClassifier(), {"learning_rate": 1}),
        (LogisticRegression(max_iter=3500, solver='saga'), {"C": 0.01}),
        (LogisticRegression(max_iter=3500, solver='saga'), {"C": 0.1}),
        (LogisticRegression(max_iter=3500, solver='saga'), {"C": 1})
    ]
)

# Convert results to DataFrame and save to CSV
graduation_multiverse_df = pd.DataFrame(results)
graduation_multiverse_df.to_csv("graduation_multiverse_result.csv", index=False)

In [84]:
# grad_df = pd.read_csv('graduation_multiverse_result.csv', low_memory=False)
# grad_df = grad_df.round(3)
# grad_df.to_csv('graduation_multiverse.csv', index=False)

In [88]:
# existing_df = pd.read_csv('partial_results.csv', low_memory=False)
# verse_dict = {
#     "Include_Transfer": 1,
#     "Handle_Nan": 'Impute',
#     "Train_Size": 0.7,
#     "Include_Sex": 1, 
#     "Include_Race": 1,
#     "Encoder": 'OneHotEncoder', 
#     "Scaler":  None,
#     "Sampler": None,
#     "Classifier": 'LogisticRegression',
#     'C': 1
#     }
